In [1]:
from src import StudyGen

In [2]:
study = StudyGen(path_configuration='config.yaml', path_master='master.yaml')

In [3]:
l_gen = study.generate_all()
for gen in l_gen:
    print(gen)

# ==================================================================================================
# --- Imports
# ==================================================================================================

import math
import numpy as np
from typing import Any

# ==================================================================================================
# --- Blocks
# ==================================================================================================

def multiply_function(a: float, b: float) -> float:

    # Multiply a and b
    return a * b

def add_function(a: float, b: float) -> float:
    """Dummy docstring"""
    # Add a and b
    return a + b

def gamma_function(a: float) -> float:
    """Dummy docstring"""
    # Compute gamma function of a
    return math.gamma(a)

def save_npy_function(output: Any, path_output: str) -> None:
    # path_output = f"{output=}".split("=")[0]
    np.save(path_output, output)


# ======================================